<a href="https://colab.research.google.com/github/chhyyi/aiffel/blob/main/LMS/AIFFEL_LMS_exp13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EXPLORATION 13 아이유 팬이 좋아할만한 다른 노래 찾기

## version of packages
```bash
tensorflow-addons             0.14.0
tensorflow-datasets           4.4.0
tensorflow-estimator          2.7.0
tensorflow-gpu                2.6.0
tensorflow-hub                0.12.0
tensorflow-metadata           1.5.0
implicit                      0.4.8
```

In [2]:
!pip3 install tensorflow==2.6.0
!pip3 install keras==2.6.0
!pip3 install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip3 install implicit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
path = '/content/drive/MyDrive/colabdata/modulabs/lms_exp13/ml-1m'
import numpy as np
import scipy
import implicit
import os
import pandas as pd
rating_file_path=os.path.join(path,'ratings.dat')
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

/usr/local/lib/python3.7/dist-packages/implicit/gpu/__init__.py:14: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  f"CUDA extension is built, but disabling GPU support because of '{e}'",


,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [4]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [5]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.path.join(path,'movies.dat')
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'


# 데이터 분석

In [8]:
#print(len(movies), len(ratings))
print(movies['title'].nunique())
print(ratings['movie_id'].nunique())
print(ratings['user_id'].nunique())


#인기있는 영화 30개
temp = ratings.groupby('movie_id')['user_id'].count()
print(temp.sort_values(ascending=False).head(30))

#평점이 높은 영화
temp = ratings.groupby('movie_id')['counts'].mean()
print(temp.sort_values(ascending=False).head(30))
temp2=temp.sort_values(ascending=False).head(30).index
for i in temp2:
    print('index ',i,'의 rating 수: ',ratings.loc[ratings['movie_id']==i]['user_id'].count())
#평점이 높은 것을 보려면 count가 일정 수 이상인 것만 봐야겠네요. rating 1개만 들어간 것들이 5점에 집중돼 있습니다. 318, 858같은 것은 인기도 있고 평점도 높습니다. 제목을 확인하면
print(movies.loc[movies['movie_id']==858])
print(movies.loc[movies['movie_id']==318])

3883
3628
6039
movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64
movie_id
1830    5.000000
3607    5.000000
3800    5.000000
3280    5.000000
989     5.000000
1360    5.000000
3656    5.000000
687     5.000000
3881    5.000000
787     5.000000
1420    5.000000
1787    5.000000
1553    5.000000
3382    5.000000
3205    5.000000
3236    5.000000
572     5.000000
3172    5.000000
3233    5.000000
3245    4.800000
53      4.750000
3232    4.750000
2503    4.666667
2197    4.666667
3866    4.666667
2905    4.647059
2019    4.629690
670     4.603774
858     4.598523
318     4.596627
Name: counts, dtype: float64
index  183

## 인기있는 영화 찾아보기
 그냥 평점이 많은 것 말고 평점 평균이 높은 것도 찾아보려고 했는데 별 소득은 없었습니다.

# 내가 선호하는 영화 추가하기

In [9]:
#제가 재미있게 본 영화들을 찾아봤는데 없습니다.
#(마스터 앤드 커맨더, 리즈와 파랑새, 덩케르크, 목소리의 형태 등을 찾아봤는데요. 연도를 보면 20세기 영화가 대부분입니다.)
#그냥 more movies.dat 으로 스페이스 눌러가면서 아는 영화를 추가합시다.
#영화의 숫자를 과소평가 한 것 같습니다. 그냥 옛날 영화들을 떠올려 보겠습니다. 
#그냥 배쉬 쉘에 movies.dat | grep terminator 이런식으로 치면서 겨우 5개를 찾았습니다.
#1036 Die Hard (1988) 3
#1240 터미네이터 3
#캐스트 어웨이->없음
#2028 라이언 일병 구하기 4
#라붐 ->없음
#1580 맨인블랙
#1676 starship troopers 4
#ratings
temp=pd.DataFrame()
temp['user_id']=[6041,6041,6041,6041,6041]
temp['movie_id']=[1036,1240,2028,1580,1676]
temp['counts']=[3,3,4,3,5]
ratings=ratings.append(temp)
print(ratings.tail(10))

         user_id  movie_id  counts    timestamp
1000203     6040      1090       3  956715518.0
1000205     6040      1094       5  956704887.0
1000206     6040       562       5  956704746.0
1000207     6040      1096       4  956715648.0
1000208     6040      1097       4  956715569.0
0           6041      1036       3          NaN
1           6041      1240       3          NaN
2           6041      2028       4          NaN
3           6041      1580       3          NaN
4           6041      1676       5          NaN


# CSR Matrix

In [11]:
from scipy.sparse import csr_matrix
#ratings.reset_index(inplace=True)
num_user = max(ratings['user_id'])+1
num_movie = max(ratings['movie_id'])+1

csr_data = csr_matrix((ratings.counts, (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6042x3953 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## CSR data의 Transpose:
음악 데이터를 들을 때에는 Transpose를 취했는데 이번에는 그렇게 하지 않습니다. 왜냐하면 하고 넣어보니까 index가 영화 index보다 큰 것들이 나옵니다. implicit 버전이 달라지면서 뭐가 바뀐 건지 보려고 찾아봤는데 못찾겠습니다.

In [12]:
#모델 구성과 훈련
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)
#csr_data_transpose = csr_data.T
#csr_data_transpose

als_model.fit(csr_data)

  0%|          | 0/15 [00:00<?, ?it/s]


# 선호도 확인해보기 

In [13]:
#선호도 확인해보기 :
#1676 starship troopers와
#1682 트루먼 쇼를 봅시다. 근데 사전을 만들어야겠습니다.

similar_artist = als_model.similar_items(1676, N=15)
similar_artist


(array([1676,  316, 1527, 1876, 1391,  780,  198, 1603,  303,  196, 2094,
         442, 1590, 1356, 1320], dtype=int32),
 array([1.0000001 , 0.6139034 , 0.55412966, 0.5479362 , 0.53943807,
        0.5106307 , 0.49014887, 0.48942375, 0.47755864, 0.47220564,
        0.4620738 , 0.455844  , 0.45510632, 0.44530606, 0.44365618],
       dtype=float32))

In [14]:
idx_to_title=dict(zip(movies['movie_id'].values, movies['title'].values))
title_to_idx=dict(zip(movies['title'].values, movies['movie_id'].values))
print(idx_to_title[1682])
def similar_title_idx2title(idx:int):
    for i in als_model.similar_items(idx, N=15)[0]:
        print(idx_to_title[i])

Truman Show, The (1998)


In [15]:
print(1682)

1682


In [16]:
print(als_model.similar_items(1682, N=5)[0])

[1682  300  150  508   62]


In [36]:
sim15=list(als_model.similar_items(1676, N=15))
for i in range(15):
    try:
        print('Similarity(max=1)',idx_to_title[sim15[0][i]], 'is', sim15[1][i])
    except:
        print('found no title for movie_id: ',i)

Similarity(max=1) Starship Troopers (1997) is 1.0000001
Similarity(max=1) Stargate (1994) is 0.6139034
Similarity(max=1) Fifth Element, The (1997) is 0.55412966
Similarity(max=1) Deep Impact (1998) is 0.5479362
Similarity(max=1) Mars Attacks! (1996) is 0.53943807
Similarity(max=1) Independence Day (ID4) (1996) is 0.5106307
Similarity(max=1) Strange Days (1995) is 0.49014887
Similarity(max=1) Mimic (1997) is 0.48942375
Similarity(max=1) Quick and the Dead, The (1995) is 0.47755864
Similarity(max=1) Species (1995) is 0.47220564
Similarity(max=1) Rocketeer, The (1991) is 0.4620738
Similarity(max=1) Demolition Man (1993) is 0.455844
Similarity(max=1) Event Horizon (1997) is 0.45510632
Similarity(max=1) Star Trek: First Contact (1996) is 0.44530606
Similarity(max=1) Alienï¿½ (1992) is 0.44365618


# 추천 영화 받아보기
USER_ID 6041로 좋아하는 영화를 넣었으니까 그걸로 찾으면 됩니다. 버전차이 때문인지 안돼서 https://github.com/benfred/implicit/issues/535 을 참고해서 좀 수정했습니다.

In [68]:
user=6041
movie_recommended = als_model.recommend(user, csr_data[user], filter_already_liked_items=True)
print(movie_recommended)



(array([ 589,  110, 2916,  480, 2571, 1200,  780,  527, 1233, 1242],
      dtype=int32), array([0.46031904, 0.38979518, 0.3868574 , 0.38001212, 0.3755428 ,
       0.28830945, 0.25938582, 0.25693107, 0.2504824 , 0.25038552],
      dtype=float32))


In [69]:
for i in range(10):
    try:
        print(idx_to_title[movie_recommended[0][i]], movie_recommended[1][i])
    except:
        print('found no title for movie_id: ',i)

Terminator 2: Judgment Day (1991) 0.46031904
Braveheart (1995) 0.38979518
Total Recall (1990) 0.3868574
Jurassic Park (1993) 0.38001212
Matrix, The (1999) 0.3755428
Aliens (1986) 0.28830945
Independence Day (ID4) (1996) 0.25938582
Schindler's List (1993) 0.25693107
Boat, The (Das Boot) (1981) 0.2504824
Glory (1989) 0.25038552


In [ ]:
print(help(csr_matrix))

### supplement: movie_id에 왜 이렇게 없는 값이 많은지...?
근본적 원인규명은 못하고 그냥 transpose를 안하게 바꿨습니다. 아무튼 이 코드를 lms로 가져가보니 als_model.similar_items가 return시키는 값의 순서가 바뀌었습니다. 버전이 바뀌면서 뭐가 많이 바꼈나봐요.

In [18]:
print(max(list(movies.movie_id.values)))

3952


In [19]:
print(num_movie, num_user)

3953 6042


In [ ]:
help(AlternatingLeastSquares)

# 회고, 토의

그냥 LMS에서 할 걸 그랬나 하는 생각이 많이 들었습니다. CSR matrix 너무 헷갈리네요. 아직도 맞게 했는지 정확히 모르겠습니다.